In [9]:
!pip install shiny shinywidgets

In [10]:
from pathlib import Path

Path("firstshiny").mkdir(parents = True, exist_ok = True)

In [11]:
%%writefile firstshiny/app.py
from typing import List

from shiny import *
from shiny.types import NavSetArg
from shiny.ui import h4
import skimage
import numpy as np
from PIL import Image, ImageOps
from pathlib import Path
from shiny import App, render, ui
from shiny.types import FileInfo, ImgData, SilentException

def nav_controls(prefix: str) -> List[NavSetArg]:
    return [
        ui.nav("Time Series", "Time series visualization from year 2020: ", ui.img(src="timeseries.png", style="width: 800px;")),
        ui.nav("First 5 observations", "First five observations from the whole dataset: ", ui.img(src="five.png", style="width: 800px;")),
        ui.nav("PM2.5 By Month", "Boxplot of PM2.5 by month in year of 2020: ", ui.img(src="box2020.png", style="width: 800px;")),
        ui.nav("PM2.5 Change"," How PM2.5 has changed on average over the years from 2015 to 2022: ", ui.img(src="byyear.png", style="width: 800px;")),
        ui.nav("Heatmap", "Correlation matrix for the dataset: ", ui.img(src="heatmap.png", style="width: 800px;")),
        ui.nav("Month Trend", "The monthly seasonal trend: ", ui.img(src="month_trend.png", style="width: 800px;")),

        ui.nav_spacer(),
        
    ]

app_ui = ui.page_navbar(
    title="EDA",
    bg="#0062cc",
    inverse=True,
    id="navbar_id",
    footer=ui.div(
        {"style": "width:80%;margin: 0 auto"},
        ui.tags.style(
            """
            h4 {
                margin-top: 3em;
            }
            """
        ),
        h4("EDA"),
        ui.navset_tab(*nav_controls("navset_tab()")),
        
    )
)


def server(input, output, session):
    @output
    @render.ui
    def images() -> ui.Tag:
        img = ui.img(src="heatmap.png", style="width: 400px;")
        return img
    
www_dir = Path(__file__).parent / "www"
app = App(app_ui, server, static_assets=www_dir)  

Overwriting firstshiny/app.py


In [12]:
!shiny run --reload firstshiny/app.py

INFO:     Will watch for changes in these directories: ['/Users/zhanna.sakayeva/Downloads/firstshiny']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [67971] using StatReload
INFO:     Started server process [67973]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:53062 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53062 - "GET /timeseries.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:53063 - "GET /five.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:53064 - "GET /box2020.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:53064 - "GET /month_trend.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:53062 - "GET /byyear.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:53063 - "GET /heatmap.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:53063 - "GET / HTTP/1.1" 200 OK
INFO:     ('127.0.0.1', 53066) - "WebSocket /websocket/" [accepted]
INFO:     connection open


In [13]:
#load
import statsmodels.api as sm

OLS2 = sm.load("OLS2.pickle")
OLS2

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
a = pd.DataFrame({'const': [1],  
              'AQI_O3_4hr': [2], 
              'AQI_PM10': [3]})
OLS2.predict(a)

0   -3.845868
dtype: float64

In [15]:
%%writefile firstshiny/app2.py
from shiny import App, render, ui
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pandas as pd

app_ui = ui.page_fluid(
  # App title
  ui.panel_title("Regression Model Prediction"),

  # Sidebar layout container
  ui.layout_sidebar(

    # Sidebar inputs 
    ui.panel_sidebar(
      ui.input_slider("O3", "AQI_O3_4hr", 0, 150, 1),
      ui.input_slider("PM10", "AQI_PM10", 0, 1000, 5),
    ),

    # Output display region 
    ui.output_text_verbatim("txt"),
    
  ),
)

    # Part 2: server ----
def server(input, output, session):
  @output
  @render.text

  def txt():
    OLS2 = sm.load("OLS2.pickle")
    a = pd.DataFrame({'const': [1],  
              'AQI_O3_4hr': [input.O3()], 
              'AQI_PM10': [input.PM10()]})
    return f"The prediction of AQI_PM2.5 concentration is {OLS2.predict(a)[0]}"

app = App(app_ui, server)

Overwriting firstshiny/app2.py


In [16]:
!shiny run --reload firstshiny/app2.py

INFO:     Will watch for changes in these directories: ['/Users/zhanna.sakayeva/Downloads/firstshiny']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [67989] using StatReload
INFO:     Started server process [67991]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:53071 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53071 - "GET / HTTP/1.1" 200 OK
INFO:     ('127.0.0.1', 53074) - "WebSocket /websocket/" [accepted]
INFO:     connection open
INFO:     connection closed
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [67991]
